<a href="https://colab.research.google.com/github/yuya1201/Digital-Advertising-HW/blob/master/Python_Exercise_1_Basic_Ad_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **Load CSV**

In [0]:
# import data
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv(open('drive/My Drive/Colab Notebooks/Travel Pony Facebook.csv', 'rb'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df.head(3)

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.0,3,246,243,0,0,0,0,243,100.0,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.0,6,215,209,0,0,0,0,209,50.0,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.0,2,252,250,0,0,0,0,250,150.0,1.190476


In [0]:
# calculate cost per impression
df['costPerImpression']=df['Amount Spent (USD)']/df.Impressions

# 1. When considering 'cost per impression', **what day of the week works best?** (What day is it cheapest to generate impressions)
# **What day works worst?** (What day is most expensive)


In [0]:
#dt = arrow.get('2013-12-02', '%Y-%m-%d')
df['Start Date']=pd.to_datetime(df['Start Date'])
#df['dayweek_n']= df['Start Date'].apply(lambda x: x.dayofweek+1)
df['dayweek']= df['Start Date'].apply(lambda x: x.strftime('%a'))

In [0]:
# aggregate cost per impression by day of week using groupby
cost1=df.groupby('dayweek')['costPerImpression'].mean()
# Best day of week (cheapest)
cost1.nsmallest(1)
# Worst day of week (most expensive)
cost1.nlargest(1)

dayweek
Sat    0.002629
Name: costPerImpression, dtype: float64

dayweek
Fri    0.004097
Name: costPerImpression, dtype: float64

In [0]:
# use pivot_table
cost2=df.pivot_table(index='dayweek',aggfunc='mean').iloc[:,-2]
# Best day of week (cheapest)
cost2.nsmallest(1)
# Worst day of week (most expensive)
cost2.nlargest(1)

dayweek
Thu    2.101045
Name: Website Clicks, dtype: float64

dayweek
Sat    3.691383
Name: Website Clicks, dtype: float64

# 2. Next, **compute the correlation** (Links to an external site.) between **Amount Spent** and the following variables:
- Reach
- Frequency
- Unique Clicks
- Page Likes

In [0]:
# df correlations
cor=df.corr()
cor1=cor.iloc[[0,1,4,6],5]
cor1

Reach            0.703124
Frequency        0.130201
Unique Clicks    0.882993
Page Likes       0.757612
Name: Amount Spent (USD), dtype: float64

Which correlation is the strongest? What does that mean practically? (respond in a tweet or less)

In [0]:
cor1.nlargest(1) # strongest
# Unique clicks and costs have a very strong positive relationship. When unique clicks increase, costs also increase.

Unique Clicks    0.882993
Name: Amount Spent (USD), dtype: float64

# 3. Finally, perform a simple **multiple regression analysis** (Links to an external site.) where **Unique Clicks is the dependent variable and Reach and Frequency are the independent (predictor) variables.**


In [0]:
# build regression model
target=df['Unique Clicks']
dep=df[['Reach','Frequency']]
mod=sm.OLS(target,dep)
res=mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Wed, 13 Nov 2019   Prob (F-statistic):                        0.00
Time:                        23:53:23   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

What variable most strongly predicts unique clicks? What does that mean practically? (respond in a tweet or less)

Frequency most strongly predicts unique clicks, compared to Reach. Frequency has 3.6139 as estimated coefficient while Reach has 0.0019. One unit increasing of Frequency will increase unique clicks by about 4 units. However, one unit increasing of Reach will not affect much on unique clicks.


